Code associated with paper "Theta oscillations as a substrate for medial prefrontal-hippocampal assembly interactions"

Michele Nardin, Karola Kaefer, Federico Stella, Jozsef Csicsvari

In this notebook, we will load spiking data, prepare the matrix of covariates, fit GLM models with L2 penalty and cross validated.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stat
import statsmodels.api as sm
from sklearn.model_selection import train_test_split as tts

Scripts for fitting GLM models based on statsmodels GLM Poisson family

In [16]:
def fitGLM_lam(X,     # matrix of covariates
                Y,    # 1 X T, vector of binned spiking data
                lam): # optimal L2 penalty found through cross-validation
    T = len(Y)
    g = sm.GLM(Y, X.astype(float), family=sm.families.Poisson())
    glm=g.fit_regularized(L1_wt=0, alpha=lam)
    return glm.params

logfac = np.array([0,0]+[np.sum(np.log(np.arange(2,n))) for n in range(3,10000)])
def loglglm(X,Y,an):
    return np.mean(-np.exp(an@X) + Y*(an@X) - logfac[Y.astype(int)])

def cross_valid_sing_cell(X,Y):

    pl = [0.000001,0.00001,0.0001,0.001,0.01,0.1] # possible L2 penalties 
    llk = [] # save log-likelihoods here across 4-fold 75/25 cross validation
    for l in pl:
        llk.append([])
        # 4-fold cross validation for each cell with random 75 train / 25 test
        for jk in range(4): 
            ## randomly split train/test
            X_train,  X_test, Y_train, Y_test = tts(X,Y,test_size=0.25,shuffle=True)
            # fit GLM
            g = sm.GLM(Y_train, X_train.astype(float), family=sm.families.Poisson())
            glm=g.fit_regularized(L1_wt=0, alpha=l,cnvrg_tol=1e-05) # L2 regularize
            # measure log-likelihood on test data
            llk[-1].append(loglglm(X_test.T,Y_test,glm.params))
    best_lam = pl[np.argmax(np.mean(llk,1))] # find best lambda
    return best_lam

load data

In [3]:
# load covariates (1-hot for position, trajectory, experimental phase, speed, theta)
covariates = np.load('data/covariates.npy')

In [4]:
# check size
covariates.shape

(29908, 424)

In [5]:
# load firing
fir_CA1 = np.load('data/fir_CA1.npy')
fir_PFC = np.load('data/fir_PFC.npy')

In [6]:
# check sizes
fir_CA1.shape,fir_PFC.shape

((29908, 46), (29908, 42))

In [17]:
# fit GLM models on single cells data
# find optimal L2 penalty through cross-validation

# CA1
T, N = fir_CA1.shape
print('CA1')
CA1_glm_params = [] # parameters
CA1_glm_firing = [] # GLM expected firing rates
for cell in range(N):
    print('*',end='')
    Y = fir_CA1[:,cell]
    # prepare X: 
    # firing of other cells within the same area
    others = np.delete(fir_CA1,cell,axis=1)
    # firing history
    hist = np.concatenate([[0],Y[:-1]]).reshape([-1,1])
    # stack with other covariates (position, trajectory, exp phase, speed, theta)
    X = np.hstack([covariates, hist,others])
    # fit GLM model using cross-validation
    lam = cross_valid_sing_cell(X,Y)
    # using the best lambda, fit the model on the full data
    params = fitGLM_lam(X,Y,lam)
    # store parameters in a vector, to save
    CA1_glm_params.append(params)
    # store GLM expected firing rate
    CA1_glm_firing.append(np.exp(X@params))
    
# PFC
T, N = fir_PFC.shape
print('\n PFC')
PFC_glm_params = []
PFC_glm_firing = []
for cell in range(N):
    Y = fir_PFC[:,cell]
    print('*',end='')
    # prepare X:
    # firing of other cells
    others = np.delete(fir_PFC,cell,axis=1)
    # firing history of the neuron of interest
    hist = np.concatenate([[0],Y[:-1]]).reshape([-1,1])
    # and stack with other covariates (position, trajectory, experimental phase, speed, theta)
    X = np.hstack([covariates, hist,others])
    # find optimal lambda using cross-validation
    lam = cross_valid_sing_cell(X,Y)
    # using the best lambda, fit the model on the full data
    params = fitGLM_lam(X,Y,lam)
    # store parameters in a vector, to save
    PFC_glm_params.append(params)
    # store GLM expected firing rate
    PFC_glm_firing.append(np.exp(X@params))

CA1
**********************************************
 PFC
******************************************

In [20]:
# Save GLM parameters
np.save('fits/CA1_glm_params',CA1_glm_params)
np.save('fits/PFC_glm_params',PFC_glm_params)

In [21]:
# also save expected firing rates - we're gonna need them later to infer noise correlations
np.save('fits/CA1_glm_firing',CA1_glm_firing)
np.save('fits/PFC_glm_firing',PFC_glm_firing)